In [0]:
%tensorflow_version 2.x

In [0]:
from __future__ import print_function
from keras.callbacks import LambdaCallback
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.optimizers import RMSprop
from keras.utils.data_utils import get_file
import numpy as np
import random
import sys
import io

Using TensorFlow backend.


In [0]:
from google.colab import drive
drive.mount('/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /gdrive


In [0]:
def save_model(m,filename):
    model_json = m.to_json()
    with open("/gdrive/My Drive/colab_data/"+filename+".json", "w") as json_file:
        json_file.write(model_json)
    # serialize weights to HDF5
    m.save_weights("/gdrive/My Drive/colab_data/"+filename+".h5")
    print("Saved model to disk")

In [0]:
def load_model_weights(filename, model):
    model.load_weights("/gdrive/My Drive/colab_data/"+filename+".h5")
    print("Loaded weights from disk")
    return model

Effettua uno smoothing di una distribuzione categorica $p_i=\frac{e^{\alpha_i}}{Z}$  in  $p'_i=\frac{e^{\sqrt[s]\alpha_i}}{Z'}$

In [0]:
def smooth_distribution(probs, s=1.0):
    probs = np.asarray(probs).astype('float64')
    probs = np.log(probs) / s
    exp_probs = np.exp(probs)
    probs = exp_probs / np.sum(exp_probs)
    return probs

Estrai un valore $0,\ldots,k$ secondo la distribuzione $p_0,\ldots,p_k$

In [0]:
def sample(probs):
  sampled = np.random.multinomial(1, probs, 1)
  return np.argmax(sampled)

Predice una sequenza casuale di caratteri utilizzando la LSTM istanziata

In [0]:
def predict_random_sequence(generated_length=400, diversity=1.0):
  generated_length = generated_length
  nlines = int(generated_length/100)
  start_index = random.randint(0, len(text) - sentence_length - 1)
  sentence = text[start_index: start_index + maxlen]
  generated = "".join(sentence)
  print('Seed:\n{0:s}'.format(generated))
  for i in range(generated_length):
    occurs = np.zeros((1, sentence_length, len(chars)))
    for t, char in enumerate(sentence):
      occurs[0, t, char_indices[char]] = 1.

    probs = model.predict(occurs, verbose=0)[0]
    probs = smooth_distribution(probs,diversity)
    next_index = sample(probs)
    next_char = indices_char[next_index]
    sentence = sentence[1:] + next_char
    generated = generated+next_char
  print('Generated:')
  for i in range(nlines):
    print('{0:s}'.format(generated[i*100:(i+1)*100]))
  print('{0:s}'.format(generated[(i+1)*100:]))

Definizione funzione di call-back, invocata durante l'apprendimento

In [0]:
def on_epoch_end(epoch, _):
    save_model(model, 'lstm0')
    print()
    print('----- Generating text after Epoch: %d' % epoch)
    predict_random_sequence(generated_length=400, diversity=1.0)

In [0]:
path = get_file('nietzsche.txt', origin='https://s3.amazonaws.com/text-datasets/nietzsche.txt')
with io.open(path, encoding='utf-8') as f:
    text = f.read().lower()

print('corpus length:', len(text))

606208/600901 [==============================] - 0s 0us/step
corpus length: 600893


In [0]:
bad_chars = ['\n']
text = ''.join(i for i in text if not i in bad_chars) 
print('revised corpus length:', len(text))

Costruzione dizionario di caratteri

In [0]:
chars = sorted(list(set(text)))
print('Total chars: {0:3d}'.format(len(chars)))

Total chars:  56


Associazione indice-carattere e viceversa

In [0]:
char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

Estrai sequenze di caratteri di stessa lunghezza, parzialmente sovrapposte

In [0]:
sentence_length = 40
step = 3
# lista di sequenze estratte
sentences = []
# lista di caratteri successivi alle frasi estratte
next_chars = []
for i in range(0, len(text) - sentence_length, step):
    sentences.append(text[i: i + sentence_length])
    next_chars.append(text[i + sentence_length])
print('{0:3d} sequenze'.format(len(sentences)))

196973 sequenze


Crea matrice delle occorrenze per ogni sequenza e complessiva, di tutte le sequenze

In [0]:
occurs_seq = np.zeros((len(sentences), sentence_length, len(chars)), dtype=np.bool)
total_occurs = np.zeros((len(sentences), len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        occurs_seq[i, t, char_indices[char]] = 1
    total_occurs[i, char_indices[next_chars[i]]] = 1

In [0]:
occurs_seq.shape

(196973, 40, 56)

In [0]:
total_occurs.shape

(196973, 56)

In [0]:
state_size = 128
model = Sequential()
model.add(LSTM(state_size, input_shape=(sentence_length, len(chars))))
model.add(Dense(len(chars), activation='softmax'))

optimizer = RMSprop(0.01)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

La rete LSTM riceve in input sentence_length valori dalla sequenza considerata, a cui aggiunge state_size valori dello stato. Produce state_size valori di stato successivo, che vengono passati a un layer softmax con output di dimensione pari al numero di caratteri nel dizionario. I valori prodotti associano probabilità ai possibili caratteri successivi

In [0]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_2 (LSTM)                (None, 128)               94720     
_________________________________________________________________
dense_2 (Dense)              (None, 56)                7224      
Total params: 101,944
Trainable params: 101,944
Non-trainable params: 0
_________________________________________________________________


In [0]:
print_callback = LambdaCallback(on_epoch_end=on_epoch_end)

In [0]:
#model = load_model_weights('lstm1', model)

Loaded weights from disk


In [0]:
model.fit(occurs_seq, total_occurs, batch_size=128, epochs=1, callbacks=[print_callback])

Epoch 1/1
196973/196973 [==============================] - 228s 1ms/step - loss: 1.5768
Saved model to disk

----- Generating text after Epoch: 0
Seed:
l, are not perhaps merely superficial es
Generated:
l, are not perhaps merely superficial estranscimery spought, are nymen live ismore the whope as a be
eneaster--a lacker, andlacks, dividence lets, lowmar" only to part in the regordited onthat sacripus
ed possimis.s, crefungten, sto veryod and internance. theremegorybeing init ward. ye atdance, forces
,in the oudmisude man in action, missuffical as it is give the philosobarding onely them impossivald
 therefo-ageted backwas condemptions as 


In [0]:
save_model(model, 'lstm1')

Saved model to disk


In [0]:
predict_random_sequence(generated_length=800, diversity=0.4)

Seed:
 pronounced a person of whomsociety shou
Generated:
 pronounced a person of whomsociety should the world to the world, one would be are the same to the 
power, in the thought of fact and the really and the individual to the contemptationalistic and are 
the value and in many the learn and in the asservation of the most present the converted there is  i
n the the real and instinct of some processis of the man, there is the world of the self persages ar
e all the sade the subtler the contempory the possession of the tempt the case of itself the confuse
 the shame of the yet desire to the thereby the such man are not the constitution and the degrated a
nd an all and the same really and and regard of and such a well the constrain to the world and the l
acker our feer and also in the world of the contempt the intellectual result the learn as a playes t
o and provery to the present the world t
